# TigerGraph and Plotly Lab

## Create your TigerGraph Graph

### Create a Solution on https://tgcloud.io/

Follow [this blog](https://www.tigergraph.com/blog/getting-started-with-tigergraph-3-0/) for complete details on starting with TG Cloud. Here are the main steps to execute:

First, go to https://tgcloud.io/ and log in or sign up if you haven’t already. Next, navigate to the “My Solutions” tab. There, on the top right corner, click “Create Solution.”

![Image](https://media.discordapp.net/attachments/691840155325038592/862564176432988200/1PX3QRwjapcj17KNECsj1hg.png)
![Image](https://media.discordapp.net/attachments/691840155325038592/862564306440028180/Screen_Shot_2021-07-08_at_00.22.32.png)

On the first page, click “Blank” then continue.

![Image](https://cdn.discordapp.com/attachments/691840155325038592/862566114815246336/1aGJxmaJZ9B11Uzooh8a07Q.png)

Don’t modify any of the settings on the second section. This will just create a free TigerGraph instance.

![Image](https://media.discordapp.net/attachments/691840155325038592/862566137153060894/1kaBtSxAgm73Yh6b75rGeag.png)

On the third section, update the credentials appropriately and keep note of the subdomain and password, as we will use this in the Python portion!

![Image](https://media.discordapp.net/attachments/691840155325038592/862566160711811092/14g6cySygGHJJ2jG2j4jvsw.png)

Finally, on the last page, verify that all of the details are correct, then press submit. This might take a few minutes to complete.

![Image](https://media.discordapp.net/attachments/691840155325038592/862566188117655552/1cTwiqRAyR9iCThPMX611mA.png)

### Install and Import pyTigerGraph

In [ ]:
!pip install pyTigerGraph

Replace SUBDOMAIN and PASSWORD with the values you assigned when creating the box.

In [ ]:
import pyTigerGraph as tg 

conn = tg.TigerGraphConnection(host="https://SUBDOMAIN.i.tgcloud.io/", password="PASSWORD")

### Create the Schema and Graph

In [ ]:
print(conn.gsql('''
CREATE VERTEX Colour(PRIMARY_ID colour_id STRING, colour_name STRING, rgb STRING, transparent STRING)
CREATE VERTEX Inventory(PRIMARY_ID inventory_id STRING, inventory_num INT)
CREATE VERTEX Version(PRIMARY_ID version_id INT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Lego_Set(PRIMARY_ID set_num STRING, num_parts INT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Part(PRIMARY_ID part_num STRING, part_name STRING)
CREATE VERTEX Part_Category(PRIMARY_ID part_cat_num STRING, part_name STRING)
CREATE VERTEX Theme(PRIMARY_ID theme_id STRING, theme_name STRING, theme_number INT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Year(PRIMARY_ID year INT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"

CREATE UNDIRECTED EDGE INVENTORY_VERSION(FROM Inventory, TO Version)
CREATE UNDIRECTED EDGE INVENTORY_LEGO_SET(FROM Inventory, TO Lego_Set, quantity INT)
CREATE UNDIRECTED EDGE LEGO_SET_THEME(FROM Lego_Set, TO Theme)
CREATE UNDIRECTED EDGE LEGO_SET_YEAR(FROM Lego_Set, TO Year)
CREATE UNDIRECTED EDGE INVENTORY_PART(FROM Inventory, TO Part, quantity INT, spare STRING)
CREATE UNDIRECTED EDGE PART_COLOUR(FROM Part, TO Colour)
CREATE UNDIRECTED EDGE PART_CATEGORY_PART(FROM Part_Category, TO Part)
CREATE DIRECTED EDGE PARENT_THEME(FROM Theme, TO Theme)
'''))

The vertex type Colour is created.
The vertex type Inventory is created.
The vertex type Version is created.
The vertex type Lego_Set is created.
The vertex type Part is created.
The vertex type Part_Category is created.
The vertex type Theme is created.
The vertex type Year is created.
The edge type INVENTORY_VERSION is created.
The edge type INVENTORY_LEGO_SET is created.
The edge type LEGO_SET_THEME is created.
The edge type LEGO_SET_YEAR is created.
The edge type INVENTORY_PART is created.
The edge type PART_COLOUR is created.
The edge type PART_CATEGORY_PART is created.
The edge type PARENT_THEME is created.


In [ ]:
print(conn.gsql('''
CREATE GRAPH LegoGraph(
  Colour, Inventory, Version, Lego_Set, Part, Part_Category, Theme, Year,
  INVENTORY_VERSION, INVENTORY_LEGO_SET, LEGO_SET_THEME, LEGO_SET_YEAR, INVENTORY_PART, PART_COLOUR, PART_CATEGORY_PART, PARENT_THEME
)
'''))

Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 12.487 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.156 seconds
The graph LegoGraph is created.


### Update the Credentials

In [ ]:
conn.graphname = "LegoGraph"
conn.apiToken = conn.getToken(conn.createSecret())

## Load Data

### Grab Kaggle Data





In [ ]:
! pip install -q kaggle

You'll need to grab Kaggle data. First, create a Kaggle account at https://www.kaggle.com/. Next, press your profile picture on the top right then click “Account.”

![Image](https://cdn.discordapp.com/attachments/691840155325038592/862661674401660938/1o5BUxUG3KYWdDsviQT1EMA.png)

Scroll down to “API” and then press “Expire API Token” then “Create New API Token.”

![Image](https://miro.medium.com/max/4800/1*3ePfyIPwCtTe0jyvV0w-Mw.png)

This will download a kaggle.json file to upload in the cell below. 




In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"genericp3rson","key":"21e2863522d868224af01d1d6bfbfc0f"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets list

ref                                                           title                                              size  lastUpdated          downloadCount  
------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                   Reddit Vaccine Myths                              233KB  2021-07-07 19:18:09           8957  
crowww/a-large-scale-fish-dataset                             A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           5483  
imsparsh/musicnet-dataset                                     MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1974  
dhruvildave/wikibooks-dataset                                 Wikibooks Dataset                                   2GB  2021-07-03 18:37:20           2371  
promptcloud/careerbuilder-job-listing-2020                    Ca

Here, we'll grab and unzip the Lego Database data.

In [ ]:
! kaggle datasets download -d rtatman/lego-database
! unzip lego-database.zip -d train
! ls train

  0% 0.00/2.94M [00:00<?, ?B/s]
100% 2.94M/2.94M [00:00<00:00, 98.8MB/s]
Archive:  lego-database.zip
  inflating: train/colors.csv        
  inflating: train/downloads_schema.png  
  inflating: train/inventories.csv   
  inflating: train/inventory_parts.csv  
  inflating: train/inventory_sets.csv  
  inflating: train/part_categories.csv  
  inflating: train/parts.csv         
  inflating: train/sets.csv          
  inflating: train/themes.csv        
colors.csv	      inventory_parts.csv  parts.csv
downloads_schema.png  inventory_sets.csv   sets.csv
inventories.csv       part_categories.csv  themes.csv


### Loading Data

We'll read the CSVs and convert them to DataFrames. Then we'll upsert them to our graph using upsertVertexDataFrame and upsertEdgeDataFrame. 

In [ ]:
import pandas as pd 

colours = pd.read_csv("train/colors.csv")
parts = pd.read_csv("train/parts.csv")
lego_sets = pd.read_csv("train/sets.csv")
themes = pd.read_csv("train/themes.csv")
inventories = pd.read_csv("train/inventories.csv")
inventory_parts = pd.read_csv("train/inventory_parts.csv")
inventory_sets = pd.read_csv("train/inventory_sets.csv")
part_categories = pd.read_csv("train/part_categories.csv")

In [ ]:
conn.upsertVertexDataFrame(colours, "Colour", "id", attributes={"colour_name": "name", "rgb": "rgb", "transparent": "is_trans"})
conn.upsertVertexDataFrame(parts, "Part", "part_num", attributes={"part_name": "name"})
conn.upsertVertexDataFrame(lego_sets, "Lego_Set", "set_num", attributes={"set_num": "set_num", "num_parts": "num_parts"})
conn.upsertVertexDataFrame(themes, "Theme", "id", attributes={"theme_number": "id", "theme_name": "name"})
conn.upsertVertexDataFrame(lego_sets, "Year", "year", attributes={"year": "year"})
conn.upsertVertexDataFrame(inventories, "Inventory", "id", attributes={"inventory_num": "id"})
conn.upsertVertexDataFrame(inventories, "Version", "version", attributes={"version_id": "version"})
conn.upsertVertexDataFrame(part_categories, "Part_Category", "id", attributes={"part_name": "name"})

conn.upsertEdgeDataFrame(inventories, "Inventory", "INVENTORY_VERSION", "Version", "id", "version", attributes={})
conn.upsertEdgeDataFrame(lego_sets, "Lego_Set", "LEGO_SET_THEME", "Theme", "set_num", "theme_id", attributes={})
conn.upsertEdgeDataFrame(parts, "Part_Category", "PART_CATEGORY_PART", "Part", from_id="part_cat_id", to_id="part_num", attributes={})
conn.upsertEdgeDataFrame(lego_sets, "Lego_Set", "LEGO_SET_YEAR", "Year", "set_num", "year", attributes={})
conn.upsertEdgeDataFrame(themes, "Theme", "PARENT_THEME", "Theme", "name", "parent_id", attributes={})
conn.upsertEdgeDataFrame(inventories, "Inventory", "INVENTORY_LEGO_SET", "Lego_Set", "id", "set_num", attributes={})
conn.upsertEdgeDataFrame(inventories, "Inventory", "INVENTORY_VERSION", "Version", "id", "version", attributes={})
conn.upsertEdgeDataFrame(inventory_parts, "Inventory", "INVENTORY_PART", "Part", "inventory_id", "part_num", attributes={"quantity": "quantity", "spare": "is_spare"})
conn.upsertEdgeDataFrame(inventory_parts, "Part", "PART_COLOUR", "Colour", "part_num", "color_id", attributes={})
conn.upsertEdgeDataFrame(inventory_sets, "Inventory", "INVENTORY_LEGO_SET", "Lego_Set", "inventory_id", "set_num", attributes={"quantity": "quantity"})


2846

## Plotly Express + TigerGraph

Now let's jump into the Plotly! We'll be using Plotly Express, Plotly's high level API for creating visuals. 

In [ ]:
import pandas as pd 
import plotly.express as px 

Here, we'll write a query. This will return the years and the number of parts in each Lego Set that year. 

In [ ]:
year_parts = conn.runInterpretedQuery('''
INTERPRET QUERY () FOR GRAPH LegoGraph {
  TYPEDEF TUPLE <year INT, num_parts INT> YEAR_PARTS;

  ListAccum<YEAR_PARTS> @@result; 

  Seed = {Year.*};

  Res = SELECT l FROM Seed:s - (LEGO_SET_YEAR:e) - Lego_Set:l
        ACCUM @@result+=YEAR_PARTS(s.year, l.num_parts);

  PRINT @@result;
}
''')

We'll change it into a DataFrame.

In [ ]:
df = pd.DataFrame(year_parts[0]["@@result"])

df

,year,num_parts
0,1953,1
1,1953,12
2,1953,5
3,1953,48
4,1971,126
...,...,...
11668,2008,129
11669,2008,31
11670,2008,5
11671,2008,0


Here, we'll scatter it.

In [ ]:
fig = px.scatter(df, x="year", y="num_parts")
fig.show()

Let's add colour using the theme. 

In [ ]:
year_parts = conn.runInterpretedQuery('''
INTERPRET QUERY () FOR GRAPH LegoGraph SYNTAX v2 {
  TYPEDEF TUPLE <year INT, num_parts INT, theme_id INT> YEAR_PARTS;

  ListAccum<YEAR_PARTS> @@result;

  Seed = {Year.*};

  Res = SELECT l FROM Seed:s - (LEGO_SET_YEAR:e) - Lego_Set:l - (LEGO_SET_THEME:e1) - Theme:t
        ACCUM @@result+=YEAR_PARTS(s.year, l.num_parts, t.theme_number);

  PRINT @@result;
}
''')

In [ ]:
df = pd.DataFrame(year_parts[0]["@@result"])

df.head()

,year,num_parts,theme_id
0,1953,48,371
1,1953,5,371
2,1953,12,371
3,1953,1,371
4,1971,57,416


In [ ]:
fig = px.scatter(df, x="year", y="num_parts", color="theme_id")
fig.show()

Great! Finally, we'll explore some more visualisations with the data.

In [ ]:
year_parts = conn.runInterpretedQuery('''
INTERPRET QUERY () FOR GRAPH LegoGraph SYNTAX v2 {
  TYPEDEF TUPLE <year INT, num_parts INT, theme_id INT, theme_name STRING> YEAR_PARTS;

  ListAccum<YEAR_PARTS> @@result;

  Seed = {Year.*};

  Res = SELECT l FROM Seed:s - (LEGO_SET_YEAR:e) - Lego_Set:l - (LEGO_SET_THEME:e1) - Theme:t
        ACCUM @@result+=YEAR_PARTS(s.year, l.num_parts, t.theme_number, t.theme_name);

  PRINT @@result;
}
''')

In [ ]:
df = pd.DataFrame(year_parts[0]["@@result"])

df.head()

,year,num_parts,theme_id,theme_name
0,1953,5,371,Supplemental
1,1953,1,371,Supplemental
2,1953,48,371,Supplemental
3,1953,12,371,Supplemental
4,1971,307,433,Minitalia


In [ ]:
fig = px.scatter(df, x="year", y="num_parts", color="theme_id", hover_data=["theme_name"])
fig.show()

In [ ]:
fig = px.scatter_3d(df, x="year", y="num_parts", z="theme_id", hover_data=["theme_name"])
fig.show()

In [ ]:
fig = px.density_heatmap(df, x="year", y="num_parts")
fig.show()

In [ ]:
fig = px.density_heatmap(df, x="year", y="theme_name")
fig.show()

In [ ]:
year_parts = conn.runInterpretedQuery('''
INTERPRET QUERY () FOR GRAPH LegoGraph SYNTAX v2 {
  TYPEDEF TUPLE <year INT, num_parts INT, quantity INT, inventory_num INT> YEAR_PARTS;

  ListAccum<YEAR_PARTS> @@result;

  Seed = {Year.*};

  Res = SELECT l FROM Seed:s - (LEGO_SET_YEAR:e) - Lego_Set:l - (INVENTORY_LEGO_SET:e1) - Inventory:t
        ACCUM @@result+=YEAR_PARTS(s.year, l.num_parts, e1.quantity, t.inventory_num);

  PRINT @@result;
}
''')

df = pd.DataFrame(year_parts[0]["@@result"])

df.head()

,year,num_parts,quantity,inventory_num
0,2011,54,0,345
1,2011,3,0,13158
2,2011,4,0,10284
3,2011,465,1,9943
4,2011,18,0,10592


In [ ]:
fig = px.box(df, x="year", y="inventory_num")
fig.show()

In [ ]:
fig = px.scatter_3d(df, x="year", y="quantity", z="num_parts", color="inventory_num")
fig.show()

In [ ]:
fig = px.scatter_3d(df, x="year", y="num_parts", z="inventory_num", size="quantity")
fig.show()

## Congrats!

In [ ]:
### IF YOU WANT TO DELETE EVERYTHING IN YOUR GRAPH, UN-COMMENT THIS.

# conn.gsql('DROP ALL')